In [ ]:
import GPy
import numpy as np
%matplotlib qt
import matplotlib.pyplot as plt

In [ ]:
def f1(x, noise=True):
    n=x.size
    if noise:
        return np.sin(x) + 0.1 * np.random.randn(n).reshape(x.shape)  
    else:
        return np.sin(x)
def f2(x, noise=True):
    n=x.size
    if noise:
        return np.cos(x) + 0.1 * np.random.randn(n).reshape(x.shape)
    else:
        return np.cos(x)

In [ ]:
# Generate training data
n = 100
x_train = np.sort(np.random.uniform(0., np.pi, n)).reshape(-1,1)

# Fit a GP model to each function
kernel = GPy.kern.RBF(input_dim=1, variance=1., lengthscale=1.)
m = GPy.models.GPRegression(x_train, f1(x_train), kernel)
m.rbf.lengthscale.constrain_bounded(0.1, 10)
m.optimize_restarts(num_restarts=2)

# Make predictions using the GPs
fig, ax = plt.subplots(1, 1)
x_query = np.linspace(0, np.pi, n).reshape(-1, 1)
for id, (f, color) in enumerate(zip([f1, f2], ['r', 'b'])):
    # Fit a GP model to the function
    y_train = f(x_train)
    m.set_XY(x_train, y_train)
    
    # Generate predictions
    mu_pred, var_pred = m.predict(x_query)
    stdev_pred = np.sqrt(var_pred)
    
    # Plot predictions
    ax.plot(x_train, y_train, '.', color='gray')
    l, = ax.plot(x_query, mu_pred, color=color, label='GP ' + str(id) + ', mean')
    ax.fill(np.concatenate([x_query, x_query[::-1]]), 
            np.concatenate([(mu_pred + 3. * stdev_pred),
                            (mu_pred - 3. * stdev_pred)[::-1]]), 
            alpha=0.25, color=l.get_color(), label=r'GP ' + str(id) + ', $3\sigma$')
x_sample = np.random.uniform(1.5, 2.0, 20)
ax.plot(x_sample, f2(x_sample), 'k*', ms=10)    
ax.set_xlabel('x')
ax.set_ylabel('g')
ax.set_title('Experience Recommendation Example')
ax.legend(loc='lower left')
ax.set_xlim(0., np.pi)
plt.show()

In [ ]:
np.sqrt(m.rbf.variance), np.sqrt(m.Gaussian_noise.variance)

In [ ]:
m